In [26]:
from raphtory import Graph
import random
from datetime import datetime, timedelta
import altair as alt
import pandas as pd


In [27]:
device_graph = Graph()

In [28]:
# Start and end dates
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 1, 1)
start_str = start_date.strftime("%Y-%m-%d %H:%M:%S")

with open("data/devices.csv", 'r') as device_file:
    next(device_file) # skip the header
    for device in device_file:
        #extract all of the information
        device = device.split(",")
        neId = device[0]
        code = device[1]
        name = device[2]
        lat = float(device[3])
        long = float(device[4])
        #build the properties as a dictionary
        properties = {"code":code,"name":name,"lat":lat,"long":long}
        #Adding the vertex at the start of the time period
        device_graph.add_vertex(timestamp=start_str,id=neId)
        #As the properties currently never change add them as 'static' so they are accessible at any time
        device_graph.add_vertex_properties(id=neId,properties=properties)


with open("data/links.csv", 'r') as links_file:
    next(links_file) # skip the header
    for link in links_file:
        #extract all of the information
        link = link.split(",")
        src=link[0]
        dst=link[1]
        link_id=link[2]
        capacity=int(link[3])
        #build the properties as a dictionary
        properties = {"link_id":link_id,"capacity":capacity}
        #Adding the edge at the start of the time period - add it both ways
        device_graph.add_edge(timestamp=start_str,src=src,dst=dst) 
        device_graph.add_edge(timestamp=start_str,src=dst,dst=src) 
        #As the properties currently never change add them as 'static' so they are accessible at any time
        device_graph.add_edge_properties(src=src,dst=dst,properties=properties)
        device_graph.add_edge_properties(src=dst,dst=src,properties=properties)

device_graph

Graph(number_of_edges=58, number_of_vertices=15, earliest_time=1577836800000, latest_time=1577836800000)

In [29]:
#available links for traffic to travel down
known_links = list(device_graph.edges())
# Generate traffic on random links - 10 per minute over the sample period
current_date = start_date
while current_date < end_date:
    time = current_date.strftime("%Y-%m-%d %H:%M:%S")
    for edge in random.sample(known_links,10):
        # add the randomly generated traffic to the edge at the current_time
        traffic = {"traffic":random.randint(1, 80)}
        device_graph.add_edge(time,edge.src().id(),edge.dst().id(),traffic)
    
    # Increment the time by 1 minute
    current_date += timedelta(minutes=1)

In [30]:
#A quick function to sum up the property values on the edge 
def edge_weight(edge,prop_name):
    return sum(list(map(lambda e: e[1],edge.property_history(prop_name))))

#Get the NO1 -> NO2 edge and create a rolling daily window  - this returns an iterator of views of the edge, one for each day in the sample period
rolling_edge = device_graph.edge("N01","N02").rolling(step='1 day', window='1 day')

#Create a new dataframe
result = pd.DataFrame()
#Run over the iterator and calculate the daily total traffic for NO1->NO2
result['Traffic Sent (MB)'] = [edge_weight(e,"traffic") for e in rolling_edge]
#Run over the iterator and get the timestamp for each edge view
result['Time'] = [e.end_date_time() for e in rolling_edge]

#Add this data to a altair chart
chart = alt.Chart(result).mark_line().encode(
    x="Time:T",
    y="Traffic Sent (MB):Q",
).properties(
    width=800,
)

#Get the capacity of this edge
capacity = device_graph.edge("N01","N02")["capacity"]
#Wrap this in a dataframe as its what the altair lib requires
capacity = pd.DataFrame({'capacity': [capacity]})
#draw this capacity as a straight line on the chart so we can see when the edge was over capacity
capacity_line = alt.Chart(capacity).mark_rule(color='red', strokeWidth=3,strokeDash=[3, 3]).encode(y='capacity:Q')

alt.layer(chart,capacity_line)


alt.LayerChart(...)